## Load Sample Ships

In [3]:
import os
import zipfile
import shutil
from pathlib import Path
import tempfile

ZIPS_PATH = "C:/Users/tiago/Documents/Workspace/UFG/pfc-cc-2025/data/zip_files"
TILES_PATH = "C:/Users/tiago/Documents/Workspace/UFG/pfc-cc-2025/data/tile_files"

os.makedirs(TILES_PATH, exist_ok=True)

temp_dir = tempfile.mkdtemp()


zip_files = [f for f in os.listdir(ZIPS_PATH) if f.endswith('.zip')]

if not zip_files:
    print(f"Nenhum arquivo ZIP encontrado em {ZIPS_PATH}")
    exit(1)

print(f"Encontrados {len(zip_files)} arquivos ZIP")

for zip_file in zip_files:
    zip_path = os.path.join(ZIPS_PATH, zip_file)
    print(f"\nProcessando: {zip_file}")
    
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            extract_folder = os.path.join(temp_dir, zip_file.replace('.zip', ''))
            zip_ref.extractall(extract_folder)
            
            gpkg_files = list(Path(extract_folder).rglob("*.gpkg"))
            
            print(f"  Encontrados {len(gpkg_files)} arquivos GPKG")
            
            for gpkg_path in gpkg_files:
                tile_id = gpkg_path.name.split('_')[0]
                
                if tile_id:
                    new_name = f"tile_{tile_id}.gpkg"
                    new_path = os.path.join(TILES_PATH, new_name)
                    
                    shutil.copy2(str(gpkg_path), new_path)
                    print(f"    ✓ {tile_id} -> {new_name}")
                else:
                    print(f"    ⚠ Não foi possível extrair número de: {gpkg_path.name}")
            
            shutil.rmtree(extract_folder)
            
    except Exception as e:
        print(f"  ❌ Erro ao processar {zip_file}: {str(e)}")

print(f"\n✅ Processamento concluído!")
print(f"Arquivos GPKG salvos em: {TILES_PATH}")

saved_files = [f for f in os.listdir(TILES_PATH) if f.endswith('.gpkg')]
print(f"Total de arquivos GPKG salvos: {len(saved_files)}")

Encontrados 5 arquivos ZIP

Processando: Auditados-20250211T092821Z-001.zip
  Encontrados 972 arquivos GPKG
    ✓ 1103 -> tile_1103.gpkg
    ✓ 1156 -> tile_1156.gpkg
    ✓ 119 -> tile_119.gpkg
    ✓ 1242 -> tile_1242.gpkg
    ✓ 1245 -> tile_1245.gpkg
    ✓ 1415 -> tile_1415.gpkg
    ✓ 159 -> tile_159.gpkg
    ✓ 185 -> tile_185.gpkg
    ✓ 1881 -> tile_1881.gpkg
    ✓ 2179 -> tile_2179.gpkg
    ✓ 2523 -> tile_2523.gpkg
    ✓ 259 -> tile_259.gpkg
    ✓ 2629 -> tile_2629.gpkg
    ✓ 2733 -> tile_2733.gpkg
    ✓ 2782 -> tile_2782.gpkg
    ✓ 2785 -> tile_2785.gpkg
    ✓ 2807 -> tile_2807.gpkg
    ✓ 2870 -> tile_2870.gpkg
    ✓ 2974 -> tile_2974.gpkg
    ✓ 3083 -> tile_3083.gpkg
    ✓ 3084 -> tile_3084.gpkg
    ✓ 311 -> tile_311.gpkg
    ✓ 3172 -> tile_3172.gpkg
    ✓ 3344 -> tile_3344.gpkg
    ✓ 3487 -> tile_3487.gpkg
    ✓ 3488 -> tile_3488.gpkg
    ✓ 3495 -> tile_3495.gpkg
    ✓ 3515 -> tile_3515.gpkg
    ✓ 3599 -> tile_3599.gpkg
    ✓ 3778 -> tile_3778.gpkg
    ✓ 37 -> tile_37.gpkg
    ✓ 3

## Load AlphaEarth Data

In [1]:
import ee

ee.Authenticate()
ee.Initialize(project="ee-tiagogoncalves")

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


In [2]:
import os
import geopandas as gpd

TILES_PATH = "C:/Users/tiago/Documents/Workspace/UFG/pfc-cc-2025/data/tile_files"
FEATURE_PATH = "C:/Users/tiago/Documents/Workspace/UFG/pfc-cc-2025/data/feature_files"

os.makedirs(FEATURE_PATH, exist_ok=True)

In [3]:
def get_download_url(geometry, year):
    left_bottom = [geometry[0], geometry[1]]
    right_top = [geometry[2], geometry[3]]

    geometry = ee.Geometry.Rectangle([left_bottom, right_top])

    s_date = ee.Date.fromYMD(year, 1, 1)
    e_date = s_date.advance(1, 'year')
                
    collection = ee.ImageCollection("GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL")
    
    image = collection.filterBounds(geometry).filterDate(s_date, e_date).first()
    
    return image.getDownloadURL({
        'scale': 10,
        'region': geometry,
        'format': 'GEO_TIFF'
    })


def download_image(idx, url):
    import requests
    response = requests.get(url)

    output_filename = f"feature_{idx}.tif"
    output_path = os.path.join(FEATURE_PATH, output_filename)
    
    if response.status_code == 200:
        with open(output_path, 'wb') as f:
            f.write(response.content)
        print(f"    ✓ Salvo: {idx}")
    else:
        print(f"    ❌ Erro ao baixar: Status {response.status_code}")


def get_gpkg_geometry(gpkg_path):
    try:
        gdf = gpd.read_file(gpkg_path)
        
        if gdf.crs != "EPSG:4326":
            gdf = gdf.to_crs("EPSG:4326")
        
        return gdf.total_bounds
                
    except Exception as e:
        print(f"❌ Erro ao processar {gpkg_path}: {str(e)}")

In [ ]:
from time import sleep

gpkg_files = [f for f in os.listdir(TILES_PATH) if f.endswith('.gpkg')]

if not gpkg_files:
    print(f"Nenhum arquivo GPKG encontrado em {TILES_PATH}")
    exit(1)

print(f"Encontrados {len(gpkg_files)} arquivos GPKG para processar")

for gpkg_file in gpkg_files:
    gpkg_path = os.path.join(TILES_PATH, gpkg_file)

    geometry = get_gpkg_geometry(gpkg_path)

    for _ in range(2):
        try:
            download_url = get_download_url(geometry, 2024)

            download_image(gpkg_file.split('_')[1].split('.')[0], download_url)

            sleep(0.25)

            break
        except Exception as e:
            sleep(2)

print("✅ Processamento concluído!")

saved_files = [f for f in os.listdir(FEATURE_PATH) if f.endswith('.tif')]
print(f"Total de arquivos GeoTIFF salvos: {len(saved_files)}")

Encontrados 4021 arquivos GPKG para processar
    ✓ Salvo: 1
    ✓ Salvo: 10
    ✓ Salvo: 10000
    ✓ Salvo: 1001
    ✓ Salvo: 1002
    ✓ Salvo: 1003
    ✓ Salvo: 1005
    ✓ Salvo: 1007
    ✓ Salvo: 1008
    ✓ Salvo: 1009
    ✓ Salvo: 101
    ✓ Salvo: 1011
    ✓ Salvo: 1015
    ✓ Salvo: 1019
    ✓ Salvo: 1020
    ✓ Salvo: 1023
    ✓ Salvo: 1026
    ✓ Salvo: 1027
    ✓ Salvo: 1028
    ✓ Salvo: 1029
    ✓ Salvo: 103
    ✓ Salvo: 1030
    ✓ Salvo: 1031
    ✓ Salvo: 1033
    ✓ Salvo: 1036
    ✓ Salvo: 1042
    ✓ Salvo: 1043
    ✓ Salvo: 1045
    ✓ Salvo: 1047
    ✓ Salvo: 1049
    ✓ Salvo: 1051
    ✓ Salvo: 1052
    ✓ Salvo: 1055
    ✓ Salvo: 1057
    ✓ Salvo: 1059
    ✓ Salvo: 1063
    ✓ Salvo: 1064
    ✓ Salvo: 1067
    ✓ Salvo: 1068
    ✓ Salvo: 107
    ✓ Salvo: 1070
    ✓ Salvo: 1071
    ✓ Salvo: 1072
    ✓ Salvo: 1073
    ✓ Salvo: 1074
    ✓ Salvo: 1076
    ✓ Salvo: 1077
    ✓ Salvo: 108
    ✓ Salvo: 1080
    ✓ Salvo: 1082
    ✓ Salvo: 1083
    ✓ Salvo: 1085
    ✓ Salvo: 1086
    ✓ Sa

## Load Satelite Data